## HW 3a: Implement Perceptron for Heart Disease

1. Use step function and $\eta = 0.1$ and batch size = 4 (ok to use 1 if need be)
2. Use TUNE set to choose ‘early stopping’ epoch for each of 10 test folds (maxEpochs = 10,000 but can stop if / when all trainset examples correct)
3. Ok to just use ONE tune set (but feel free to use 9 as done in HW1 and HW2)
4. Once epochsToUse estimated, train on all 9 folds; use (8/9) epochsToUse
5. Report best epoch and min, mean, and max accuracy per train-tune fold 
6. Compare to Random Forests and kNN test set min, mean, and max

In [1]:
import numpy as np

def step_function(x: int):
    return 1 if x >= 0 else 0

class Perceptron:
    def __init__(self, num_weights, activation_function, learning_rate=0.1):
        self.weights = np.zeros(num_weights + 1)
        self.activation_function = activation_function
        self.learning_rate = learning_rate

    def fit(self, X: np.array, y: np.array): # X is 3-d array, y is 2-d array with batch size in first dimension
        for X_batch, y_batch in zip(X, y):
            # print(f'weights before: {self.weights}')
            updates = []
            assert X_batch.shape[0] == y_batch.shape[0]
            for example, y in zip(X_batch, y_batch):
                example = np.append(example, -1)
                y_bar = self.predict_one(example)
                # print(y_bar)
                update = self.learning_rate * (y - y_bar) * example
                # print(f'example: {example}, update: {update}')
                updates.append(update)
            self.weights = self.weights + np.sum(updates, axis=0) / X_batch.shape[0]
            # print(f'weights after: {self.weights}')

    def predict_one(self, example: np.array):
        return self.activation_function(self.weights.T @ example)

    def predict(self, examples: np.array):
        output = []
        for example in examples:
            example = np.append(example, -1)
            output.append(self.predict_one(example))
        return np.array(output)

In [2]:
# unit tests

clf = Perceptron(4, step_function)
x = np.array([[[1, 2, 3, 1], [2, 3, 4, 1], [2, 3, 4, 1]], [[5, 6, 7, 1], [8, 9, 10, 1], [2, 3, 4, 1]]])
y = np.array([[1, 0, 1], [0, 0, 1]])

clf.fit(x, y)

In [21]:
from typing import List
from tqdm import trange

def accuracy_score(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return (y_true == y_pred).sum() / len(y_true)

def load_one_file(filename: str):
    data = np.load(filename, allow_pickle=True)
    X = data['x']
    y = data['y']
    example_names = data['example_names']
    return X, y, example_names

def load_folds(folds: List[int]):
    all_X = []
    all_y = []
    all_example_names = []
    for fold in folds:
        filename = f'../hw0/data/heart_fold{fold}.npz'
        X, y, example_names = load_one_file(filename)
        all_X.append(X)
        all_y.append(y)
        all_example_names.append(example_names)

    X_folds = np.concatenate(all_X, axis=0)
    y_folds = np.concatenate(all_y)
    example_names_folds = np.concatenate(all_example_names)
        
    return X_folds, y_folds, example_names_folds

X, y, example_names = load_folds([0, 1])
X.shape

(60, 13)

In [22]:
def create_batched_data(X: np.array, y: np.array, batch_size):
    assert len(X.shape) == 2 and len(y.shape) == 1
    # losing some data here
    num_batches = X.shape[0] // batch_size
    X = X[:num_batches * batch_size]
    y = y[:num_batches * batch_size]
    X = X.reshape(-1, batch_size, X.shape[1])
    y = y.reshape(-1, batch_size)

    return X, y

X, y = create_batched_data(np.ones((7, 3)), np.ones(7), 2)
X.shape, y.shape

((3, 2, 3), (3, 2))

In [5]:
import pandas as pd
import time

def get_tune_results(train_tune_folds: List[int], batch_size=4, MAX_EPOCHS=5000): # TODO: set batch size 1 for debugging 
    for tune_fold in range(len(train_tune_folds)):
        X_tune, y_tune, example_names_tune = load_folds([tune_fold])
        train_folds = [fold for fold in train_tune_folds if fold != tune_fold]
        X_train, y_train, example_names_train = load_folds(train_folds)

        clf = Perceptron(X_train.shape[1], step_function)
        best_epoch = 0
        best_accuracy = 0
        for i in trange(MAX_EPOCHS):
            perm = np.random.permutation(X_train.shape[0])
            X_train = X_train[perm]
            y_train = y_train[perm]
            X_batched, y_batched = create_batched_data(X_train, y_train, batch_size=batch_size)
            clf.fit(X_batched, y_batched)
            y_pred_train = clf.predict(X_train)
            train_accuracy = accuracy_score(y_train, y_pred_train)
            if train_accuracy == 1:
                print(f"achieved perfect train accuracy at epoch {i}")
                break
            y_pred = clf.predict(X_tune)
            tune_accuracy = accuracy_score(y_tune, y_pred)
            if tune_accuracy > best_accuracy:
                best_epoch = i
                best_accuracy = tune_accuracy
                # print(best_accuracy, best_epoch)
            # if i % 1000 == 0:
            #     print(f'epoch {i}: ')
            #     print(np.abs(clf.weights).sum(), np.abs(clf.weights).max(), np.abs(clf.weights).min()) # print min max sum of absolute value of weights per epoch
            # print absolute value of weight updates
            # okay try eta / 10, eta / 100
        break
        
    print(f'best epoch: {best_epoch}')
    print(f'last train accuracy: {train_accuracy}')
    print(f'best tune accuracy: {best_accuracy}')
    return best_epoch

def cv(num_folds=10):
    cv_results = []
    ks = []
    for test_fold in range(num_folds):
        print(f'starting fold {test_fold}')
        start = time.time()
        X_test, y_test, example_names_test = load_folds([test_fold])
        train_folds = [fold for fold in range(num_folds) if fold != test_fold]
        best_epoch = get_tune_results(train_folds)
        X_train, y_train, example_names_train = load_folds(train_folds)
        clf = Perceptron(X_train.shape[1], step_function)
        for i in range(int(best_epoch * (8/9))):
            perm = np.random.permutation(X_train.shape[0])
            X_train = X_train[perm]
            y_train = y_train[perm]
            X_batched, y_batched = create_batched_data(X_train, y_train, batch_size=4)
            clf.fit(X_batched, y_batched)
        y_pred = clf.predict(X_test)
        print(y_pred)
        acc = accuracy_score(y_test, y_pred)
        cv_results.append(acc)
        end = time.time()
        print(f'time for fold: {end - start}')
        # break

    print(ks)
    return cv_results

cv_scores = cv()
print(f'average cv score: {np.mean(cv_scores)}')

starting fold 0


FileNotFoundError: [Errno 2] No such file or directory: '/Users/brwang/Desktop/ml_class/hw0/data/heart_fold0.npz'

In [ ]:
print(f'min cv score: {np.min(cv_scores)}')
print(f'max cv score: {np.max(cv_scores)}')

# before bugfix
# min cv score: 0.6551724137931034
# max cv score: 0.9333333333333333

min cv score: 0.6551724137931034
max cv score: 0.9333333333333333


# HW 3b

1. Repeat above but use one layer of 128 ReLU hidden units & backprop
2. Use Sigmoid as the activation function for the output
3. If maxEpochs = 10,000 runs too slowly, use 5,000 or 1,000
4. Optional: try #HUs in {16, 32, 64, 128, 256, 512, 1024, …, YouChooseMax} and use the best #HUs + stopping epoch on tune set per train-test fold
5. Compare to results discussed in HW3a

In [62]:
import numpy as np
import math

class NeuralNetwork:
    def __init__(self, eta=0.01):
        kaiming_std = np.sqrt(2/13) # kaiming initialization
        self.hidden_weights = np.random.normal(0, kaiming_std, size=(128, 13))
        self.hidden_bias = np.random.normal(0, kaiming_std, size=128)
        self.output_weights = np.random.normal(0, kaiming_std, size=128)
        self.output_bias = np.random.normal(0, kaiming_std)
        self.backprop_state = {}
        self.eta = eta

    def relu(self, x): # vectorized function
        return np.maximum(x, 0) 

    def sigmoid(self, x): # vectorized function
        return 1 / (1 + np.exp(-x)) 

    def fit(self, X: np.array, y: np.array): # X is 3-d array, y is 2-d array with batch size in first dimension
        for X_batch, y_batch in zip(X, y):
            # print(f'weights before: {self.weights}')
            total_hidden_update = np.zeros((128, 13))
            total_output_update = np.zeros(128)
            total_hidden_bias_update = np.zeros(128)
            total_output_bias_update = 0

            assert X_batch.shape[0] == y_batch.shape[0]
            for example, y in zip(X_batch, y_batch):
                y_bar = self.forward_prop(example)
                hidden_update, output_update, hidden_error, output_error = self.back_prop(example, y)
                total_hidden_update += hidden_update
                total_output_update += output_update
                total_hidden_bias_update += hidden_error
                total_output_bias_update += output_error

            self.hidden_weights -= total_hidden_update / X_batch.shape[0]
            self.output_weights -= total_output_update / X_batch.shape[0]
            self.hidden_bias -= total_hidden_bias_update / X_batch.shape[0]
            self.output_bias -= total_output_bias_update / X_batch.shape[0]
            # print(f'weights after: {self.weights}')

    def back_prop(self, x, y):
        output_error = (self.backprop_state['a_L'] - y)*(1 - self.backprop_state['a_L'])*self.backprop_state['a_L']
        output_update = self.backprop_state['a_L-1'] * output_error

        hidden_error = output_error * self.output_weights * self.backprop_state['sigprime(z_L-1)']
        broadcasted_example = np.broadcast_to(x, (128, 13))
        hidden_update = np.broadcast_to(hidden_error, (13, 128)).T # * broadcasted_example

        return hidden_update, output_update, hidden_error, output_error


    def forward_prop(self, x):
        x = self.hidden_weights @ x + self.hidden_bias
        self.backprop_state['z_L-1'] = x
        self.backprop_state['sigprime(z_L-1)'] = np.where(x > 0, 1, 0)
        x = self.relu(x)
        self.backprop_state['a_L-1'] = x
        x = self.output_weights @ x + self.output_bias
        self.backprop_state['z_L'] = x
        x = self.sigmoid(x)
        self.backprop_state['a_L'] = x

        # print(self.backprop_state)
        return x 


    def predict(self, examples: np.array):
        output = []
        for example in examples:
            output.append(self.forward_prop(example))
        return np.array(output)


clf = NeuralNetwork()
clf.forward_prop(np.ones(13))

0.23413753346060412

In [81]:
# fit to random vector

x = np.random.normal(size=(1, 1, 13))
y = np.ones((1, 1))
for i in range(1000):
    clf.fit(x, y)
    if (i % 100) == 0:
        print(f'{i}: {y[0] - clf.forward_prop(x[0][0])}')

0: [1.6872583e-06]
100: [1.68725827e-06]
200: [1.68725824e-06]
300: [1.68725821e-06]
400: [1.68725817e-06]
500: [1.68725814e-06]
600: [1.68725811e-06]
700: [1.68725808e-06]
800: [1.68725804e-06]
900: [1.68725801e-06]


In [96]:
def normalize(X: np.array, means=None, std=None):
    if means is None or std is None:
        means = X.mean(axis=0)
        std = X.std(axis=0)

    return (X - means) / std, means, std


def min_max_normalize(X: np.array):
    return (X - X.min()) / (X.max() - X.min())

In [91]:
# try on 1 fold

X_train, y_train, _ = load_folds([0])
X_train, means, std = normalize(X_train)
# X_train = min_max_normalize(X_train)

X_train.max(), X_train.min()

(3.263676837555066, -2.8574475022613934)

In [92]:
X_batched, y_batched = create_batched_data(X_train, y_train, batch_size=4)
X_batched.shape

(7, 4, 13)

In [94]:
clf = NeuralNetwork()
best_epoch = 0
best_accuracy = 0
for i in trange(1000):
    # print(X_batched.shape)
    clf.fit(X_batched, y_batched)
    y_pred_train = clf.predict(X_train)
    y_pred_train = np.round(y_pred_train)
    train_accuracy = accuracy_score(y_train, y_pred_train)
    if i % 100 == 0:
        # print(y_pred_train)
        print(f'{i}: {train_accuracy}')

 16%|█▋        | 164/1000 [00:00<00:01, 820.67it/s]

0: 0.6
100: 0.9666666666666667


 33%|███▎      | 330/1000 [00:00<00:00, 808.61it/s]

200: 0.9666666666666667
300: 0.9666666666666667


 49%|████▉     | 491/1000 [00:00<00:00, 788.70it/s]

...

 73%|███████▎  | 731/1000 [00:00<00:00, 789.90it/s]

600: 0.9666666666666667
700: 0.9666666666666667


 89%|████████▉ | 893/1000 [00:01<00:00, 799.53it/s]

800: 0.9666666666666667
900: 0.9666666666666667


100%|██████████| 1000/1000 [00:01<00:00, 797.33it/s]


In [100]:
import pandas as pd
import time

def get_tune_results(train_tune_folds: List[int], batch_size=4, MAX_EPOCHS=1000): # TODO: set batch size 1 for debugging 
    for tune_fold in range(len(train_tune_folds)):
        X_tune, y_tune, example_names_tune = load_folds([tune_fold])
        train_folds = [fold for fold in train_tune_folds if fold != tune_fold]
        X_train, y_train, example_names_train = load_folds(train_folds)

        X_train, means, std = normalize(X_train)
        X_tune, _, _ = normalize(X_tune, means=means, std=std)

        print(f'Xtune: {X_tune.shape}')

        clf = NeuralNetwork()
        best_epoch = 0
        best_accuracy = 0
        for i in trange(MAX_EPOCHS):
            perm = np.random.permutation(X_train.shape[0])
            X_train = X_train[perm]
            y_train = y_train[perm]
            X_batched, y_batched = create_batched_data(X_train, y_train, batch_size=batch_size)
            # print(X_batched.shape)
            clf.fit(X_batched, y_batched)
            y_pred_train = clf.predict(X_train)
            y_pred_train = np.round(y_pred_train)
            train_accuracy = accuracy_score(y_train, y_pred_train)
            if train_accuracy == 1:
                print(f"achieved perfect train accuracy at epoch {i}")
                break
            y_pred = clf.predict(X_tune)
            y_pred = np.round(y_pred)
            tune_accuracy = accuracy_score(y_tune, y_pred)
            if tune_accuracy > best_accuracy:
                best_epoch = i
                best_accuracy = tune_accuracy
                # print(best_accuracy, best_epoch)
            # if i % 1000 == 0:
            #     print(f'epoch {i}: ')
            #     print(np.abs(clf.weights).sum(), np.abs(clf.weights).max(), np.abs(clf.weights).min()) # print min max sum of absolute value of weights per epoch
            # print absolute value of weight updates
            # okay try eta / 10, eta / 100
        break
        
    print(f'best epoch: {best_epoch}')
    print(f'last train accuracy: {train_accuracy}')
    print(f'best tune accuracy: {best_accuracy}')
    return best_epoch

def cv(num_folds=10):
    cv_results = []
    ks = []
    for test_fold in range(num_folds):
        print(f'starting fold {test_fold}')
        start = time.time()
        X_test, y_test, example_names_test = load_folds([test_fold])
        train_folds = [fold for fold in range(num_folds) if fold != test_fold]
        best_epoch = get_tune_results(train_folds)
        X_train, y_train, example_names_train = load_folds(train_folds)

        X_train, means, std = normalize(X_train)
        X_test, _, _ = normalize(X_test, means=means, std=std)
        clf = NeuralNetwork()
        for i in range(int(best_epoch * (8/9))):
            perm = np.random.permutation(X_train.shape[0])
            X_train = X_train[perm]
            y_train = y_train[perm]
            X_batched, y_batched = create_batched_data(X_train, y_train, batch_size=4)
            clf.fit(X_batched, y_batched)
        y_pred = clf.predict(X_test)
        y_pred = np.round(y_pred)
        acc = accuracy_score(y_test, y_pred)
        print(f'test acc {test_fold}: {acc}')
        cv_results.append(acc)
        end = time.time()
        print(f'time for fold: {end - start}')
        # break

    print(ks)
    return cv_results



cv()

starting fold 0
Xtune: (30, 13)


100%|██████████| 1000/1000 [00:11<00:00, 89.13it/s]
/var/folders/lr/597pxhmx2z78r4vysc_dvsth0000gn/T/ipykernel_82293/481716268.py:18: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


best epoch: 21
last train accuracy: 0.9101123595505618
best tune accuracy: 0.8666666666666667
test acc 0: 0.6333333333333333
time for fold: 11.384541273117065
starting fold 1
Xtune: (30, 13)


100%|██████████| 1000/1000 [00:10<00:00, 98.91it/s] 


best epoch: 86
last train accuracy: 0.9282700421940928
best tune accuracy: 0.8666666666666667
test acc 1: 0.4666666666666667
time for fold: 10.736838340759277
starting fold 2
Xtune: (30, 13)


 14%|█▍        | 143/1000 [00:01<00:08, 98.52it/s]


KeyboardInterrupt: 